In [ ]:
#Create a team 1 win column to determine which rows actually have them winning
pdf['team1_win'] = (pdf['team1_score']>pdf['team2_score']).astype(int)

# Random sample of rows with columns in training columns list
pdf_1 = pdf.sample(frac=.5, random_state=15)[training_columns]
pdf_1 = pdf_1.reset_index(drop=True)

#Make the remainder of rows into the second df
pdf_2 = pdf[~pdf.index.isin(pdf_1.index)][training_columns].reset_index(drop=True)

#In The second df, inverve the odds and seed differential, as well as make team 1 the loser
pdf_2['team1_log5'] = 1-pdf_2['team1_log5']
pdf_2['seed_diff'] = pdf_2['seed_diff']*-1
pdf_2['team1_win'] = 0

#Remake The Game Id in the second df
pdf_2['game_id'] = (pdf_2['game_id'].str.split('-', expand=True)[0] + '-' + pdf_2['game_id'].str.split('-', expand=True)[2] + '-' + pdf_2['game_id'].str.split('-', expand=True)[1])

#Limit Second df to the columns of interest
pdf_2.columns = training_columns

#Combine the two dfs and perform a train test split
#Create pdf_adj, which is now balanced when team 1 wins
pdf_adj = pd.concat([pdf_2, pdf_1]).reset_index(drop=True)

train, test = train_test_split(pdf_adj, train_size=.7)
train = train.reset_index(drop=True)
test = test.reset_index(drop=True)

#Filter 2019 Training Data
train = train[train['season'] != 2019]
test = test[test['season'] != 2019]

In [ ]:
train, test = train_test_split(pdf_adj, train_size=.7)
train = train.reset_index(drop=True)
test = test.reset_index(drop=True)

#Filter 2019 Training Data
train = train[train['season'] != 2019]
test = test[test['season'] != 2019]

In [ ]:
#Split Training And Test Sets By Columns
X_train = train[df_features]
y_train = train[df_target]
X_test = test[df_features]
y_test = test[df_target]

#Check For Correct Shape
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)